In [23]:
import os
import re
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool
from pandas.testing import assert_frame_equal

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
lmap = lambda funcion, iterable: list(map(funcion, iterable))
lfilter = lambda funcion, iterable: list(filter(funcion, iterable))

In [4]:
dir_dict = dict()

dir_dict["data"] = "../data/"
dir_dict["raw"] = os.path.join(dir_dict["data"], "raw")
dir_dict["mid"] = os.path.join(dir_dict["data"], "mid")
dir_dict["raw_csv"] = os.path.join(dir_dict["raw"], "csv")
dir_dict["raw_json"] = os.path.join(dir_dict["raw"], "json")
dir_dict["html"] = os.path.join(dir_dict["raw"], "html")
dir_dict["completed_html"] = os.path.join(dir_dict["html"], "completed")
dir_dict["upcoming_html"] = os.path.join(dir_dict["html"], "upcoming")
dir_dict["fighterlist_html"] = os.path.join(dir_dict["html"], "fighterlist")
dir_dict["fighters_html"] = os.path.join(dir_dict["html"], "fighters")
dir_dict["completed_eventlist_html"] = os.path.join(dir_dict["completed_html"], "eventlist")
dir_dict["completed_events_html"] = os.path.join(dir_dict["completed_html"], "events")
dir_dict["completed_fights_html"] = os.path.join(dir_dict["completed_html"], "fights")
dir_dict["upcoming_eventlist_html"] = os.path.join(dir_dict["upcoming_html"], "eventlist")
dir_dict["upcoming_events_html"] = os.path.join(dir_dict["upcoming_html"], "events")
dir_dict["upcoming_fights_html"] = os.path.join(dir_dict["upcoming_html"], "fights")

In [5]:
filepath = os.path.join(dir_dict["mid"], "completed_fights.csv")
df = pd.read_csv(filepath)

/tmp/ipykernel_190278/2262833221.py:2: DtypeWarning: Columns (217,219,220) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


In [6]:
df.shape

(6875, 257)

In [7]:
df.head()

,Event Name,Fight ID,Fighter1 Status,Fighter1 Name,Fighter2 Status,Fighter2 Name,Bout,Method,Round,Time,Time format,Referee,Details,Fighter1_Overall_KD,Fighter2_Overall_KD,Fighter1_Round1_KD,Fighter2_Round1_KD,Fighter1_Round2_KD,Fighter2_Round2_KD,Fighter1_Overall_Sig. str.,Fighter2_Overall_Sig. str.,Fighter1_Round1_Sig. str.,Fighter2_Round1_Sig. str.,Fighter1_Round2_Sig. str.,Fighter2_Round2_Sig. str.,Fighter1_Overall_Sig. str. %,Fighter2_Overall_Sig. str. %,Fighter1_Round1_Sig. str. %,Fighter2_Round1_Sig. str. %,Fighter1_Round2_Sig. str. %,Fighter2_Round2_Sig. str. %,Fighter1_Overall_Total str.,Fighter2_Overall_Total str.,Fighter1_Round1_Total str.,Fighter2_Round1_Total str.,Fighter1_Round2_Total str.,Fighter2_Round2_Total str.,Fighter1_Overall_Td,Fighter2_Overall_Td,Fighter1_Round1_Td,Fighter2_Round1_Td,Fighter1_Round2_Td,Fighter2_Round2_Td,Fighter1_Overall_Td %,Fighter2_Overall_Td %,Fighter1_Round1_Td %,Fighter2_Round1_Td %,Fighter1_Round2_Td %,Fighter2_Round2_Td %,Fighter1_Overall_Sub. att,Fighter2_Overall_Sub. att,Fighter1_Round1_Sub. att,Fighter2_Round1_Sub. att,Fighter1_Round2_Sub. att,Fighter2_Round2_Sub. att,Fighter1_Overall_Rev.,Fighter2_Overall_Rev.,Fighter1_Round1_Rev.,Fighter2_Round1_Rev.,Fighter1_Round2_Rev.,Fighter2_Round2_Rev.,Fighter1_Overall_Ctrl,Fighter2_Overall_Ctrl,Fighter1_Round1_Ctrl,Fighter2_Round1_Ctrl,Fighter1_Round2_Ctrl,Fighter2_Round2_Ctrl,Fighter1_Overall_Sig. str_SS,Fighter2_Overall_Sig. str_SS,Fighter1_Round1_Sig. str_SS,Fighter2_Round1_Sig. str_SS,Fighter1_Round2_Sig. str_SS,Fighter2_Round2_Sig. str_SS,Fighter1_Overall_Sig. str. %_SS,Fighter2_Overall_Sig. str. %_SS,Fighter1_Round1_Sig. str. %_SS,Fighter2_Round1_Sig. str. %_SS,Fighter1_Round2_Sig. str. %_SS,Fighter2_Round2_Sig. str. %_SS,Fighter1_Overall_Head_SS,Fighter2_Overall_Head_SS,Fighter1_Round1_Head_SS,Fighter2_Round1_Head_SS,Fighter1_Round2_Head_SS,Fighter2_Round2_Head_SS,Fighter1_Overall_Body_SS,Fighter2_Overall_Body_SS,Fighter1_Round1_Body_SS,Fighter2_Round1_Body_SS,Fighter1_Round2_Body_SS,Fighter2_Round2_Body_SS,Fighter1_Overall_Leg_SS,Fighter2_Overall_Leg_SS,Fighter1_Round1_Leg_SS,Fighter2_Round1_Leg_SS,Fighter1_Round2_Leg_SS,Fighter2_Round2_Leg_SS,Fighter1_Overall_Distance_SS,Fighter2_Overall_Distance_SS,Fighter1_Round1_Distance_SS,Fighter2_Round1_Distance_SS,Fighter1_Round2_Distance_SS,Fighter2_Round2_Distance_SS,Fighter1_Overall_Clinch_SS,Fighter2_Overall_Clinch_SS,Fighter1_Round1_Clinch_SS,Fighter2_Round1_Clinch_SS,Fighter1_Round2_Clinch_SS,Fighter2_Round2_Clinch_SS,Fighter1_Overall_Ground_SS,Fighter2_Overall_Ground_SS,Fighter1_Round1_Ground_SS,Fighter2_Round1_Ground_SS,Fighter1_Round2_Ground_SS,Fighter2_Round2_Ground_SS,Fighter1_Round3_KD,Fighter2_Round3_KD,Fighter1_Round3_Sig. str.,Fighter2_Round3_Sig. str.,Fighter1_Round3_Sig. str. %,Fighter2_Round3_Sig. str. %,Fighter1_Round3_Total str.,Fighter2_Round3_Total str.,Fighter1_Round3_Td,Fighter2_Round3_Td,Fighter1_Round3_Td %,Fighter2_Round3_Td %,Fighter1_Round3_Sub. att,Fighter2_Round3_Sub. att,Fighter1_Round3_Rev.,Fighter2_Round3_Rev.,Fighter1_Round3_Ctrl,Fighter2_Round3_Ctrl,Fighter1_Round3_Sig. str_SS,Fighter2_Round3_Sig. str_SS,Fighter1_Round3_Sig. str. %_SS,Fighter2_Round3_Sig. str. %_SS,Fighter1_Round3_Head_SS,Fighter2_Round3_Head_SS,Fighter1_Round3_Body_SS,Fighter2_Round3_Body_SS,Fighter1_Round3_Leg_SS,Fighter2_Round3_Leg_SS,Fighter1_Round3_Distance_SS,Fighter2_Round3_Distance_SS,Fighter1_Round3_Clinch_SS,Fighter2_Round3_Clinch_SS,Fighter1_Round3_Ground_SS,Fighter2_Round3_Ground_SS,Fighter1_Round4_KD,Fighter2_Round4_KD,Fighter1_Round4_Sig. str.,Fighter2_Round4_Sig. str.,Fighter1_Round4_Sig. str. %,Fighter2_Round4_Sig. str. %,Fighter1_Round4_Total str.,Fighter2_Round4_Total str.,Fighter1_Round4_Td,Fighter2_Round4_Td,Fighter1_Round4_Td %,Fighter2_Round4_Td %,Fighter1_Round4_Sub. att,Fighter2_Round4_Sub. att,Fighter1_Round4_Rev.,Fighter2_Round4_Rev.,Fighter1_Round4_Ctrl,Fighter2_Round4_Ctrl,Fighter1_Round4_Sig. str_SS,Fighter2_Round4_Sig. str_SS,Fighter1_Round4_Sig. str. %_

In [8]:
df.isnull().sum()

Event Name               0
Fight ID                 0
Fighter1 Status          0
Fighter1 Name            0
Fighter2 Status          0
                      ... 
Fighter2 Str. Def     5173
Fighter2 TD Avg.      5173
Fighter2 TD Acc.      5173
Fighter2 TD Def.      5173
Fighter2 Sub. Avg.    5173
Length: 257, dtype: int64

### Rename columns

In [9]:
df.columns = [col.lower().replace(".","").replace(" ", "_") for col in df.columns]

In [28]:
to_clean_cols = df.columns

In [29]:
def remove_cleaned_from_list(done_cols):
    global to_clean_cols
    to_clean_cols = [col for col in to_clean_cols if col not in done_cols]
    print(f"Remaining columns: {len(to_clean_cols)}")

### ID columns

In [30]:
col_names = [col for col in df.columns if "_id" in col]
col_names

['fight_id', 'event_id', 'fighter1_id', 'fighter2_id']

In [31]:
df[col_names].head()

,fight_id,event_id,fighter1_id,fighter2_id
0,0730c4e6fb247fa1,883922e5cd6d8473,d156513a19acf856,2f181c0467965b98
1,55833bf5054126e2,0ee783aa00e468f0,6f018c039b4d5d80,30a09e43f15f1d75
2,f19a56ecc6c150e3,8788beb528894f33,6c2030e0a143a94b,3591d0d5d382a381
3,0bdea30d7a7322f7,db1f2ed63b54b9a7,5e4eec08896c9423,4d44f37107f01b9b
4,acec5f4c6fee1ae2,8a9c6c4301f6d088,008ea710276c9606,31bb0772f21cabd8


In [32]:
assert_frame_equal(df[col_names], df[col_names].applymap(lambda s: hex(int(str(s), 16))[2:].zfill(16)))

In [35]:
df.loc[:, col_names] = df[col_names].applymap(lambda s: int(str(s), 16))

In [37]:
df[col_names].head()

,fight_id,event_id,fighter1_id,fighter2_id
0,518130453482602401,9815915233220068467,15084333311904774230,3393493124467022744
1,6161834638714152674,1073971802333800688,7998828360305040768,3503974524743785845
2,17409322884500902115,9766265477231365939,7791281096687331659,3860095972428391297
3,855300257531241207,15789390316237535655,6795628409158145059,5567842706126805915
4,12460439050927348450,9987977108652478600,40153134515656198,3583466118772992984
